# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [101]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

# Import API key
from api_keys import geoapify_key

In [125]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:/Users/DeMiko Birden/Downloads/python-api-challenge/Starter_Code/cities.csv")

# Display sample data
city_data_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kuacjok,8.3002,27.9903,74.07,87,100,3.33,SS,1697671616
1,1,kirakira,-10.4544,161.9205,85.35,69,83,10.04,SB,1697671519
2,2,sambava,-14.2667,50.1667,72.28,94,23,5.66,MG,1697671616
3,3,tura,25.5198,90.2201,67.05,90,0,3.02,IN,1697671616
4,4,pitkyaranta,61.5734,31.4792,35.51,80,96,6.35,RU,1697671616


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [126]:
%%capture --no-display

# Dependencies
import folium

# Create a map centered at a specific location (e.g., latitude 0, longitude 0)
m = folium.Map(location=[0, 0], zoom_start=2)  

# Iterate through the rows of city_data_df
for index, row in city_data_df.iterrows():
    # Get the latitude, longitude, and humidity values
    lat, lon, humidity = row['Lat'], row['Lng'], row['Humidity']
    
    # Create a circle marker with size based on humidity
    folium.CircleMarker(
        location=[lat, lon],
        radius=humidity/10,  # Adjust the scale factor as needed
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f'City: {row["City"]}, Humidity: {humidity}%'
    ).add_to(m)

# Display the map
m


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [122]:
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")




In [123]:
city_data_df = pd.read_csv("output_data/cities.csv", delimiter=';')


In [127]:
print(city_data_df.columns)


Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')


In [131]:
# Narrow down cities that fit criteria and drop any results with null values

max_temp_range = (70, 80)  # Fahrenheit
max_humidity = 70  # Percentage

# Filter the DataFrame
ideal_conditions_df = city_data_df[
    (city_data_df['Max Temp'] >= max_temp_range[0]) &
    (city_data_df['Max Temp'] <= max_temp_range[1]) &
    (city_data_df['Humidity'] < max_humidity)
]

# Display sample data
(ideal_conditions_df.head)



<bound method NDFrame.head of      City_ID              City      Lat       Lng  Max Temp  Humidity  \
67        67       west valley  46.5921 -120.6051     71.78        39   
83        83        mooloolaba -26.6833  153.1167     72.05        63   
85        85            zaranj  30.9667   61.8833     72.18        26   
92        92        san julian  21.0167 -102.1667     71.06        22   
120      120         kununurra -15.7667  128.7333     76.98        50   
123      123            safaga  26.7292   33.9365     78.39        43   
124      124        las palmas  28.4204  -14.0131     72.39        69   
145      145         centralia  46.7162 -122.9543     74.77        59   
163      163             kifri  34.6959   44.9661     73.40        35   
177      177            avarua -21.2078 -159.7750     77.05        53   
203      203           zambezi -13.5432   23.1047     73.51        49   
204      204          beinamar   8.6698   15.3813     76.87        65   
213      213         

### Step 3: Create a new DataFrame called `hotel_df`.

In [130]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()



# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
(hotel_df.head())

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kuacjok,SS,8.3002,27.9903,87,
1,kirakira,SB,-10.4544,161.9205,69,
2,sambava,MG,-14.2667,50.1667,94,
3,tura,IN,25.5198,90.2201,90,
4,pitkyaranta,RU,61.5734,31.4792,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [135]:
import requests

# Set parameters to search for a hotel
radius = 10000  # Example: 10 kilometers

# Define parameters
params = {
    'categories': 'hotels',  # Specify that you're interested in hotels
    'limit': 1,              # Limit the number of results to 1 (for finding the first hotel)
    'apiKey': 'api_keys.py'  # Replace with your actual Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle({lon},{lat},{radius})"
    params["bias"] = f"circle({lon},{lat},{radius})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print(hotel_df)

       


Starting hotel search
kuacjok - nearest hotel: No hotel found
kirakira - nearest hotel: No hotel found
sambava - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
pitkyaranta - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: No hotel found
nemuro - nearest hotel: No hotel found
igarka - nearest hotel: No hotel found
qamdo - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
bathsheba - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
nyurba - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
zouerate - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
albany - nearest hotel: No hotel foun

alenquer - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
palmas - nearest hotel: No hotel found
at taraf - nearest hotel: No hotel found
weno - nearest hotel: No hotel found
banda aceh - nearest hotel: No hotel found
prince rupert - nearest hotel: No hotel found
bubaque - nearest hotel: No hotel found
ancud - nearest hotel: No hotel found
sona - nearest hotel: No hotel found
zambezi - nearest hotel: No hotel found
beinamar - nearest hotel: No hotel found
tippi - nearest hotel: No hotel found
touros - nearest hotel: No hotel found
sumberpucung - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
dakhla - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
mokhsogollokh - nearest hotel: No hotel found
nova sintra - nearest hotel: No hotel found
aswan - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
tiquipaya - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
georgetown - neare

kasba tadla - nearest hotel: No hotel found
tadine - nearest hotel: No hotel found
khandyga - nearest hotel: No hotel found
nadi - nearest hotel: No hotel found
chadan - nearest hotel: No hotel found
smithers - nearest hotel: No hotel found
lieksa - nearest hotel: No hotel found
manokwari - nearest hotel: No hotel found
horta - nearest hotel: No hotel found
pundong - nearest hotel: No hotel found
dolinsk - nearest hotel: No hotel found
patterson - nearest hotel: No hotel found
timaru - nearest hotel: No hotel found
palikir - national government center - nearest hotel: No hotel found
klaksvik - nearest hotel: No hotel found
mendoza - nearest hotel: No hotel found
nabire - nearest hotel: No hotel found
fortaleza - nearest hotel: No hotel found
kudahuvadhoo - nearest hotel: No hotel found
recife - nearest hotel: No hotel found
biltine - nearest hotel: No hotel found
bag - nearest hotel: No hotel found
yenisarbademli - nearest hotel: No hotel found
priozersk - nearest hotel: No hotel found

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [139]:
%%capture --no-display

# Configure the map plot

import folium


m = folium.Map(location=[0, 0], zoom_start=2)  


for index, row in city_data_df.iterrows():
    # Get the latitude, longitude, and humidity values
    lat, lon, humidity, city, country, hotel_name = row['Lat'], row['Lng'], row['Humidity'], row['City'], row['Country'], row['Hotel Name']
    
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=humidity/10,  
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f'City: {city}, Country: {country}, Humidity: {humidity}%, Hotel: {hotel_name}'
    ).add_to(m)

# Display the map
m


KeyError: 'Hotel Name'